In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os
import mne
import torch.nn.functional as F
import datetime
from torch.utils.data import TensorDataset, DataLoader, random_split, WeightedRandomSampler

In [2]:
# Load the data
data = torch.load(r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\bcic_iv_2a\processed\2024-06-11_14-23-47\dataset_2024-06-11_14-23-47.pt")
label = torch.load(r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\bcic_iv_2a\processed\2024-06-11_14-23-47\labels_2024-06-11_14-23-47.pt")

# Create a dataset and dataloaders
dataset = TensorDataset(torch.tensor(data, dtype=torch.float32), torch.tensor(label, dtype=torch.long))
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

C:\Users\chakr\AppData\Local\Temp\ipykernel_3976\3675817226.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset = TensorDataset(torch.tensor(data, dtype=torch.float32), torch.tensor(label, dtype=torch.long))


In [12]:
# # Load the first dataset and labels
# try:
#     data1 = torch.load(r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\bcic_iv_2a\processed\2024-06-11_14-23-47\dataset_2024-06-11_14-23-47.pt")
#     label1 = torch.load(r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\bcic_iv_2a\processed\2024-06-11_14-23-47\labels_2024-06-11_14-23-47.pt")
# except FileNotFoundError:
#     print("File 1 not found. Please check the path.")
#     raise

# # Load the second dataset and labels
# try:
#     data2 = torch.load(r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\bcic_iv_2a\processed\another_date\dataset_another_date.pt")
#     label2 = torch.load(r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\bcic_iv_2a\processed\another_date\labels_another_date.pt")
# except FileNotFoundError:
#     print("File 2 not found. Please check the path.")
#     raise

# # Combine the datasets and labels
# data = torch.cat((data1, data2), dim=0)
# labels = torch.cat((label1, label2), dim=0)

# # Create a dataset
# dataset = TensorDataset(torch.tensor(data, dtype=torch.float32), torch.tensor(labels, dtype=torch.long))

# # Ensure reproducibility
# torch.manual_seed(42)

# # Split the dataset into training and validation sets
# train_size = int(0.8 * len(dataset))
# val_size = len(dataset) - train_size
# train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# # Create weights for the sampler to make the model more specific to one of the datasets
# # Assume we want to make the model more specific to dataset1
# weights = np.concatenate((
#     np.ones(len(data1)) * 0.8,  # weight 0.8 for dataset1
#     np.ones(len(data2)) * 0.2   # weight 0.2 for dataset2
# ))

# # Create a weighted random sampler
# sampler = WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)

# # Create dataloaders
# train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler)
# val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# # Check if the data is loaded correctly
# print(f"Number of training samples: {len(train_loader.dataset)}")
# print(f"Number of validation samples: {len(val_loader.dataset)}")

In [5]:
# Load the data
try:
    data = torch.load(r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\bcic_iv_2a\processed\2024-06-11_14-23-47\dataset_2024-06-11_14-23-47.pt")
    label = torch.load(r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\bcic_iv_2a\processed\2024-06-11_14-23-47\labels_2024-06-11_14-23-47.pt")
except FileNotFoundError:
    print("File not found. Please check the path.")
    raise

# Create a dataset and dataloaders
dataset = TensorDataset(torch.tensor(data, dtype=torch.float32), torch.tensor(label, dtype=torch.long))

# Ensure reproducibility
torch.manual_seed(42)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Check if the data is loaded correctly
print(f"Number of training samples: {len(train_loader.dataset)}")
print(f"Number of validation samples: {len(val_loader.dataset)}")


C:\Users\chakr\AppData\Local\Temp\ipykernel_3976\3128540443.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset = TensorDataset(torch.tensor(data, dtype=torch.float32), torch.tensor(label, dtype=torch.long))


Number of training samples: 1036
Number of validation samples: 260


In [6]:
class EEG_MI_CNN(nn.Module):
    def __init__(self):
        super(EEG_MI_CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1)
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.dropout = nn.Dropout(p=0.5)
        
        # Adjusted based on the final output size after conv layers
        self.fc1 = nn.Linear(64 * 2 * 15, 128)  
        self.fc2 = nn.Linear(128, 2)  # Assuming 2 classes for binary classification

    def _forward_conv_layers(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        return x

    def forward(self, x):
        x = self._forward_conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

In [7]:
class Advanced_EEG_MI_CNN(nn.Module):
    def __init__(self):
        super(Advanced_EEG_MI_CNN, self).__init__()
        # First convolutional layer with Batch Normalization and ReLU activation
        self.conv1 = nn.Conv2d(in_channels=8, out_channels=32, kernel_size=(3, 3), padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        # Second convolutional layer with Batch Normalization and ReLU activation
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        # Third convolutional layer with Batch Normalization and ReLU activation
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        # Dropout layer to prevent overfitting
        self.dropout = nn.Dropout(p=0.5)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 2 * 15, 256)  # Adjust based on final output size after conv layers
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 2)  # Assuming 2 classes for binary classification

    def _forward_conv_layers(self, x):
        # Forward pass through the first convolutional layer
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        # Forward pass through the second convolutional layer
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        # Forward pass through the third convolutional layer
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        return x

    def forward(self, x):
        # Forward pass through the convolutional layers
        x = self._forward_conv_layers(x)
        # Flatten the tensor to feed into fully connected layers
        x = x.view(x.size(0), -1)
        # Forward pass through the first fully connected layer with ReLU and dropout
        x = self.dropout(F.relu(self.fc1(x)))
        # Forward pass through the second fully connected layer with ReLU
        x = F.relu(self.fc2(x))
        # Output layer
        x = self.fc3(x)
        return x

In [8]:
class Wavelet_CNN(nn.Module):
    def __init__(self):
        super(Wavelet_CNN, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=8, out_channels=64, kernel_size=(3, 3), padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3, 3), padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.dropout = nn.Dropout(p=0.5)
        
        # Attention mechanism
        self.attention = nn.Sequential(
            nn.Conv2d(512, 128, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(128, 512, kernel_size=1),
            nn.Sigmoid()
        )
        
        # Fully connected layers
        self.fc1 = nn.Linear(512 * 2 * 15, 128)  # Adjusted based on the final output size after conv layers
        self.fc2 = nn.Linear(128, 2)  # Assuming 2 classes for binary classification

    def _forward_conv_layers(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        
        # Attention mechanism
        attention_weights = self.attention(x)
        x = x * attention_weights
        
        return x

    def forward(self, x):
        x = self._forward_conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x


In [9]:
class Advanced_EEG_MI_CNN_LSTM_Transformer(nn.Module):
    def __init__(self):
        super(Advanced_EEG_MI_CNN_LSTM_Transformer, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=8, out_channels=32, kernel_size=(3, 3), padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        # LSTM layer
        self.lstm = nn.LSTM(input_size=128*2*15, hidden_size=128, num_layers=1, batch_first=True)
        
        # Transformer layer
        self.transformer_encoder_layer = nn.TransformerEncoderLayer(d_model=128, nhead=8)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, num_layers=1)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 2)
        
        # Dropout layer
        self.dropout = nn.Dropout(p=0.5)
        
    def _forward_conv_layers(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        return x

    def forward(self, x):
        batch_size = x.size(0)
        # Forward pass through convolutional layers
        x = self._forward_conv_layers(x)
        
        # Flatten for LSTM
        x = x.view(batch_size, -1, 128*2*15)
        
        # Forward pass through LSTM
        x, (hn, cn) = self.lstm(x)
        
        # Transformer encoding
        x = self.transformer_encoder(x)
        
        # Take the output from the last sequence step for classification
        x = x[:, -1, :]
        
        # Forward pass through fully connected layers
        x = self.dropout(F.relu(self.fc1(x)))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import datetime
import os

# Assuming the necessary model class and dataloaders are already defined
# e.g., model = Advanced_EEG_MI_CNN(), train_loader, val_loader

# Define a function to save the model with the date and time in the filename
def save_model(model, path):
    try:
        torch.save(model.state_dict(), path)
        print(f'Model saved as {path}')
    except Exception as e:
        print(f'Error saving the model: {e}')

# Example usage:
model = Advanced_EEG_MI_CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}')

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(val_loader.dataset)
    val_accuracy = correct / total
    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

# Save the model with the current date and time
current_time = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
model_dir = r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\trained_models\base"
os.makedirs(model_dir, exist_ok=True)
model_filename = os.path.join(model_dir, f'{model.__class__.__name__}_{current_time}.pth')
save_model(model, model_filename)

In [ ]:
# Save the model with date and time in the filename
def save_model(model, path):
    torch.save(model.state_dict(), path)

# Example usage:
model = Advanced_EEG_MI_CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}')

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(val_loader.dataset)
    val_accuracy = correct / total
    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

# Save the model with current date and time
current_time = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
model_dir = r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\trained_models\base"
print("Model directory:", model_dir)
os.makedirs(model_dir, exist_ok=True)
model_filename = os.path.join(model_dir, f'{model.__class__.__name__}_{current_time}.pth')
print("Model filename:", model_filename)
save_model(model, model_filename)
print(f'Model saved as {model_filename}')

In [13]:
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

def evaluate_model_from_paths(model_path, dataset_path, labels_path, model_class, batch_size=32, device=None):
    # Load the model architecture
    model = model_class()  # Initialize the model architecture
    model.load_state_dict(torch.load(model_path))  # Load the state dict
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Load the dataset and labels
    dataset_tensors = torch.load(dataset_path)
    labels_tensors = torch.load(labels_path)

    # Create the dataset
    dataset = CustomDataset(dataset_tensors, labels_tensors)

    # Create a DataLoader for the dataset
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    # Define a function to evaluate the model
    def evaluate_model(model, data_loader, device):
        model.eval()  # Set the model to evaluation mode
        correct = 0
        total = 0

        with torch.no_grad():  # Disable gradient calculation for evaluation
            for data, labels in data_loader:
                data, labels = data.to(device), labels.to(device)
                outputs = model(data)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        return accuracy

    # Evaluate the model
    accuracy = evaluate_model(model, data_loader, device)
    print(f'Accuracy: {accuracy * 100:.2f}%')
    return accuracy

In [53]:
# Define paths to the model, dataset, and labels
model_path = r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\trained_models\base\Advanced_EEG_MI_CNN_2024-06-10_20-29-22.pth"
data_to_predict = r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\openvibe\to_predict\openvibe_dataset_2024-06-10_20-19-04.pt"
labels_to_predict = r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\openvibe\to_predict\openvibe_labels_2024-06-10_20-19-04.pt"

# Call the function
accuracy = evaluate_model_from_paths(model_path, data_to_predict, labels_to_predict, Advanced_EEG_MI_CNN)


Accuracy: 40.00%


In [ ]:
# Define paths to the model, dataset, and labels
model_path = r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\trained_models\base\Advanced_EEG_MI_CNN_LSTM_Transformer_2024-06-10_19-56-45.pth"
dataset_path = r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\openvibe\to_predict\openvibe_dataset_2024-06-10_19-42-56.pt"
labels_path = r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\openvibe\to_predict\openvibe_labels_2024-06-10_19-42-56.pt"

# Call the function
accuracy = evaluate_model_from_paths(model_path, dataset_path, labels_path, Advanced_EEG_MI_CNN_LSTM_Transformer)